In [36]:
from sqlalchemy import create_engine, Column, Integer, String, DateTime, ForeignKey, Enum, Float
from sqlalchemy.orm import sessionmaker, declarative_base, validates
import pandas as pd

# 创建数据库连接引擎和会话工厂
engine = create_engine('sqlite:///products.db')
Session = sessionmaker(bind=engine)

# 创建模型基类
Base = declarative_base()

# 定义商品模型类
class Product(Base):
    __tablename__ = 'products'

    id = Column(Integer, primary_key=True)
    ts = Column(DateTime, nullable=False)
    name = Column(String(256), nullable=False)
    color = Column(String(64), nullable=False)
    size = Column(String(64), nullable=False)
    photo_path = Column(String(256), nullable=True)

    @validates('color')
    def validate_color(self, key, color):
        allowed_colors = ["紅", "綠", "藍", "黃", "黑", "灰"]
        if color not in allowed_colors:
            raise ValueError(f"invalid color, allowed values are {allowed_colors}")
        return color

    @validates('size')
    def validate_size(self, key, size):
        allowed_sizes = ["S", "M", "L", "2L", "XL", "XXL"]
        if size not in allowed_sizes:
            raise ValueError(f"invalid size, allowed values are {allowed_sizes}")
        return size

class Detail(Base):
    __tablename__ = 'detail'

    id = Column(Integer, ForeignKey('products.id'), primary_key=True)
    ts = Column(DateTime, default=DateTime, primary_key=True)
    quantity = Column(Integer, nullable=False)
    price = Column(Float, nullable=False)
    type = Column(Enum('IN', 'OUT', name='type'), nullable=False)
    supplier = Column(String(256), nullable=True)
    note = Column(String(256), nullable=True)


# 创建数据表
Base.metadata.create_all(engine)

# 创建商品记录的函数
def create_product(name, color, size, photo_path=None):
    session = Session()
    product = Product(ts=pd.Timestamp.now(), name=name, color=color, size=size, photo_path=photo_path)
    session.add(product)
    session.commit()
    return product

# 根据商品 ID 查找商品记录的函数
def read_product(id):
    session = Session()
    product = session.query(Product).filter_by(id=id).first()
    session.commit()
    return product

# 更新商品记录的函数
def update_product(id, name=None, color=None, size=None, photo_path=None):
    session = Session()
    product = session.query(Product).filter_by(id=id).first()
    if name:
        product.name = name
    if color:
        product.color = color
    if size:
        product.size = size
    if photo_path:
        product.photo_path = photo_path
    session.commit()
    return product

# 删除商品记录的函数
def delete_product(id):
    session = Session()
    product = session.query(Product).filter_by(id=id).first()
    session.delete(product)
    session.commit()
    return product

# 创建商品记录的函数
def create_product＿detail(quantity, price, type, supplier, note=None):
    session = Session()
    product = Product(ts=pd.Timestamp.now(), quantity=quantity, price=price, type=type, supplier=supplier, note=note)
    session.add(product)
    session.commit()
    return product

# 根据商品 ID 查找商品记录的函数
def read_product_detail(id):
    session = Session()
    product = session.query(Product).filter_by(id=id).first()
    session.commit()
    return product


In [37]:
colors = ["紅", "綠", "藍", "黃", "黑", "灰"]
size = ["S", "M", "L", "2L", "XL", "XXL"]

for c in colors:
    for s in size:
        create_product('女裝', c, s)

# create_product('女裝', '黑', 'color')

In [38]:
import pandas as pd

def get_data():
    with Session() as s:
        results = s.query(Product).all()

    df = pd.DataFrame([(p.id, p.name, p.color, p.size, p.photo_path, p.create_ts) for p in results], 
                    columns=['id', 'name', 'color', 'size', 'photo_path', 'create_ts'])
    return df

# print(df)

get_data()


,id,name,color,size,photo_path,create_ts
0,1,女裝,紅,S,None,2023-05-10 16:17:44.316409
1,2,女裝,紅,M,None,2023-05-10 16:17:44.328109
2,3,女裝,紅,L,None,2023-05-10 16:17:44.330956
3,4,女裝,紅,2L,None,2023-05-10 16:17:44.332804
4,5,女裝,紅,XL,None,2023-05-10 16:17:44.334585
5,6,女裝,紅,XXL,None,2023-05-10 16:17:44.338338
6,7,女裝,綠,S,None,2023-05-10 16:17:44.339889
7,8,女裝,綠,M,None,2023-05-10 16:17:44.344238
8,9,女裝,綠,L,None,2023-05-10 16:17:44.348121
9,10,女裝,綠,2L,None,2023-05-10 16:17:44.350826


In [24]:
delete_product(1)

In [27]:
get_data()

,id,name,color,size,photo_path
0,2,女裝,紅,S,None
1,3,女裝,紅,M,None
2,4,女裝,紅,L,None
3,5,女裝,紅,2L,None
4,6,女裝,紅,XL,None
5,7,女裝,紅,XXL,None
6,8,女裝,綠,S,None
7,9,女裝,綠,M,None
8,10,女裝,綠,L,None
9,11,女裝,綠,2L,None
